<a href="https://colab.research.google.com/github/vasudhab21/ML-LAB/blob/main/ML_Lab_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)

project_path = '/content/drive/MyDrive/GenomeDetector'
natural_data_path = os.path.join(project_path, 'data', 'natural2')
engineered_data_path = os.path.join(project_path, 'data', 'engineered')

!pip install biopython

print("Environment is ready. Paths are set to your 'natural2' and 'engineered' folders.")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.8 MB/s eta 0:00:00
Environment is ready. Paths are set to your 'natural2' and 'engineered' folders.


In [ ]:
import numpy as np
from itertools import product
from Bio import SeqIO
import glob
import os
import gzip

def get_kmer_features(k):
    """Generates all possible k-mers for a given k."""
    letters = ['A', 'C', 'G', 'T']
    return sorted([''.join(p) for p in product(letters, repeat=k)])

def sequence_to_kmer_counts(sequence, k, kmer_features):
    """Converts a DNA sequence into a vector of k-mer counts."""
    kmer_counts = {kmer: 0 for kmer in kmer_features}
    for i in range(len(sequence) - k + 1):
        kmer = sequence[i:i+k].upper()
        if kmer in kmer_counts:
            kmer_counts[kmer] += 1
    return list(kmer_counts.values())

# --- Main Loading Logic ---
print("--- Loading and Processing Your Datasets ---")
K_VALUE = 4
kmer_features = get_kmer_features(K_VALUE)
X, y = [], []

# 1. Load Natural Data from 'natural2' (handles .fna and .fna.gz)
print("Loading 'Natural' genomes from the 'natural2' folder...")
# The '*' wildcard finds files ending in .fna or .fna.gz
search_pattern_natural = f"{natural_data_path}/*.fna*"
natural_files = glob.glob(search_pattern_natural)

for filepath in natural_files:
    # Open the file correctly, whether it's gzipped or not
    handle = gzip.open(filepath, "rt") if filepath.endswith(".gz") else open(filepath, "r")
    for record in SeqIO.parse(handle, "fasta"):
        if len(record.seq) > 1000:
            X.append(sequence_to_kmer_counts(str(record.seq), K_VALUE, kmer_features))
            y.append(0) # Label 0 for Natural
    handle.close()

# 2. Load Engineered Data from 'engineered'
print("Loading 'Engineered' genomes from the 'engineered' folder...")
search_pattern_engineered = f"{engineered_data_path}/*.fasta"
engineered_files = glob.glob(search_pattern_engineered)

for filepath in engineered_files:
    for record in SeqIO.parse(filepath, "fasta"):
        if 200 < len(record.seq) < 50000:
            X.append(sequence_to_kmer_counts(str(record.seq), K_VALUE, kmer_features))
            y.append(1) # Label 1 for Engineered

X = np.array(X)
y = np.array(y)

print("\n--- Data Processing Complete! ---")
print(f"Shape of our final feature matrix X: {X.shape}")
print(f"Shape of our final label vector y: {y.shape}")
print(f"Number of Natural samples (class 0): {np.sum(y == 0)}")
print(f"Number of Engineered samples (class 1): {np.sum(y == 1)}")

--- Loading and Processing Your Datasets ---
Loading 'Natural' genomes from the 'natural2' folder...
Loading 'Engineered' genomes from the 'engineered' folder...

--- Data Processing Complete! ---
Shape of our final feature matrix X: (17168, 256)
Shape of our final label vector y: (17168,)
Number of Natural samples (class 0): 16302
Number of Engineered samples (class 1): 866


In [ ]:
# --- 1. Model Training and Comparison (A3) ---
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler

# Import all the classifiers
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

# ---- Modularized Functions ----
def evaluate_model(model, X_train, y_train, X_test, y_test):
    """Trains a model and returns its performance on train and test sets."""
    model.fit(X_train, y_train)

    # Predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Calculate metrics
    train_metrics = {
        'Train Accuracy': accuracy_score(y_train, y_pred_train),
        'Train F1-Score': f1_score(y_train, y_pred_train)
    }
    test_metrics = {
        'Test Accuracy': accuracy_score(y_test, y_pred_test),
        'Test F1-Score': f1_score(y_test, y_pred_test)
    }
    return train_metrics, test_metrics

# --- Main Program Logic ---
if 'X' not in locals() or X.shape[0] == 0:
    print("ERROR: Data variables 'X' and 'y' not found. Please run the data loading cell first.")
else:
    print("--- A3: Comparing Various Classifiers ---")

    # Some models (like SVM and MLP) work better with scaled data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

    # The "Model Zoo"
    classifiers = {
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Random Forest": RandomForestClassifier(random_state=42),
        "AdaBoost": AdaBoostClassifier(random_state=42),
        "XGBoost": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
        "CatBoost": CatBoostClassifier(random_state=42, verbose=0),
        "Naive Bayes": GaussianNB(),
        "SVM": SVC(random_state=42),
        "MLP (Neural Net)": MLPClassifier(random_state=42, max_iter=500)
    }

    results_list = []

    for name, model in classifiers.items():
        print(f"Training {name}...")
        train_metrics, test_metrics = evaluate_model(model, X_train, y_train, X_test, y_test)

        # Combine results
        result_row = {'Model': name}
        result_row.update(train_metrics)
        result_row.update(test_metrics)
        results_list.append(result_row)

    # Create and display a professional table using pandas
    results_df = pd.DataFrame(results_list)
    print("\n--- Tabulated Results ---")
    print(results_df.to_string())

    print("\n--- Observation ---")
    print("Ensemble models like Random Forest and XGBoost generally perform the best and show good generalization.")
    print("A large gap between Train F1 and Test F1 indicates potential overfitting.")

--- A3: Comparing Various Classifiers ---
Training Decision Tree...
Training Random Forest...
Training AdaBoost...
Training XGBoost...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [16:57:33] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Training CatBoost...
Training Naive Bayes...
Training SVM...
Training MLP (Neural Net)...

--- Tabulated Results ---
              Model  Train Accuracy  Train F1-Score  Test Accuracy  Test F1-Score
0     Decision Tree        1.000000        1.000000       0.992040       0.919132
1     Random Forest        1.000000        1.000000       0.994176       0.939271
2          AdaBoost        0.983440        0.820235       0.980392       0.775056
3           XGBoost        1.000000        1.000000       0.997864       0.978389
4          CatBoost        1.000000        1.000000       0.996894       0.968504
5       Naive Bayes        0.333611        0.129376       0.341487       0.128916
6               SVM        0.949571        0.000000       0.949524       0.000000
7  MLP (Neural Net)        0.994008        0.939698       0.992623       0.928571

--- Observation ---
Ensemble models like Random Forest and XGBoost generally perform the best and show good generalization.
A large gap between 

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.5 MB/s eta 0:00:00


In [ ]:
# --- 2. Hyperparameter Tuning (A2) ---
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

print("\n--- A2: Hyperparameter Tuning with RandomizedSearchCV ---")

# We'll tune the RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# Define the "dials" to turn (hyperparameters) and the range of values to try
# This is a distribution, not a fixed grid
param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': [None] + list(randint(10, 50).rvs(5)),
    'min_samples_leaf': randint(1, 5)
}

# n_iter=10 means it will try 10 random combinations
# cv=3 means it will use 3-fold cross-validation
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1 # Use all available CPU cores
)

# Run the search
random_search.fit(X_train, y_train)

print("\n--- RandomizedSearchCV Results ---")
print(f"Best Parameters Found: {random_search.best_params_}")
print(f"Best Cross-Validated F1-Score: {random_search.best_score_:.4f}")


--- A2: Hyperparameter Tuning with RandomizedSearchCV ---
Fitting 3 folds for each of 10 candidates, totalling 30 fits

--- RandomizedSearchCV Results ---
Best Parameters Found: {'max_depth': np.int64(37), 'min_samples_leaf': 1, 'n_estimators': 51}
Best Cross-Validated F1-Score: 0.9923


In [ ]:
# --- 3. Explainable AI with SHAP (O1) ---
!pip install shap

import shap

print("\n--- O1: Using SHAP for Model Explainability ---")

# We'll use the powerful XGBoost model for our explanation
# (Tree-based models are fastest with SHAP)
model_to_explain = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss').fit(X_train, y_train)

# Create the explainer
explainer = shap.TreeExplainer(model_to_explain)
# Calculate SHAP values for the test set (can take a minute)
shap_values = explainer.shap_values(X_test)

# Get the k-mer feature names for our plots
feature_names = get_kmer_features(K_VALUE)

# --- Plot 1: Global Feature Importance ---
print("\nSHAP Summary Plot (Global Feature Importance):")
print("This plot shows the most important k-mers overall. Each dot is a sample.")
print("High feature values are red, low are blue.")
shap.summary_plot(shap_values, X_test, feature_names=feature_names)

# --- Plot 2: Explanation of a Single Prediction ---
# Let's explain why the model made a specific prediction for the first "Engineered" sample in the test set
engineered_idx = np.where(y_test == 1)[0][0]

print(f"\nSHAP Force Plot for a single 'Engineered' prediction (Sample {engineered_idx}):")
print("This plot shows which features PUSHED the prediction higher (towards Engineered) and which PULLED it lower.")
shap.initjs() # Initialize javascript for plotting
shap.force_plot(
    explainer.expected_value,
    shap_values[engineered_idx, :],
    X_test[engineered_idx, :],
    feature_names=feature_names
)

In [ ]:
# --- 4. Explainable AI with LIME (O2) ---
!pip install lime

import lime
import lime.lime_tabular

print("\n--- O2: Using LIME for Local Model Explainability ---")

# LIME needs a function that takes data and returns prediction probabilities
# Our CatBoost model is a good choice here
model_to_explain_lime = CatBoostClassifier(random_state=42, verbose=0).fit(X_train, y_train)
predict_fn = lambda x: model_to_explain_lime.predict_proba(x)

# Create the LIME explainer
explainer_lime = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train,
    feature_names=feature_names,
    class_names=['Natural', 'Engineered'],
    mode='classification'
)

# --- Explanation 1: Why did the model think this was ENGINEERED? ---
engineered_idx = np.where(y_test == 1)[0][0]
print(f"\nLIME Explanation for one 'Engineered' sample (Sample {engineered_idx}):")
exp_engineered = explainer_lime.explain_instance(
    data_row=X_test[engineered_idx],
    predict_fn=predict_fn,
    num_features=6 # Show the top 6 features
)
exp_engineered.show_in_notebook(show_table=True)


# --- Explanation 2: Why did the model think this was NATURAL? ---
natural_idx = np.where(y_test == 0)[0][0]
print(f"\nLIME Explanation for one 'Natural' sample (Sample {natural_idx}):")
exp_natural = explainer_lime.explain_instance(
    data_row=X_test[natural_idx],
    predict_fn=predict_fn,
    num_features=6
)
exp_natural.show_in_notebook(show_table=True)